# Sphere function
---
Description:

- Optimization (min)
- Single-objective
- Constraints (no)
---

The general equation is given by:

- $f(\mathbf{x}) = \sum_{i=1}^{M} x^2_i$, with $-\infty \le x_i \le +\infty$

and global minimum found at:

- $f(x_1, x_2, ..., x_M) = f(0, 0, ..., 0) = 0$.

## First we import python libraries and set up the directory of our code

In [9]:
import os, sys
import numpy as np
import cProfile, pstats
from math import isclose

PROJECT_DIR = os.path.abspath('..')
sys.path.append(PROJECT_DIR)

## Here we import all our custom PSO code

In [10]:
from star_pso.auxiliary.swarm import Swarm
from star_pso.auxiliary.particle import Particle
from star_pso.engines.standard_pso import StandardPSO

## Define the sphere function

In [11]:
# Sphere function.
def fun_Sphere(x: np.typing.ArrayLike):
    
    # Compute the shpere function.
    f_val = np.sum(x**2)

    # Condition for termination.
    solution_found = isclose(f_val, 0.0, rel_tol=1.0e-6)
    
    # Return the solution tuple.
    return -f_val, solution_found
# _end_def_

## Here we set the PSO parameters

- Set the number of dimensions 'D'
- Set the number of particles 'N'
- Setup the initial population
- Create the PSO object

In [12]:
# Random number generator.
rng = np.random.default_rng()

# Define the number of optimizing variables.
D = 5

# Define the number of particles.
N = min(5*D, 100)

# Sample the initial points randomly.
X_t0 = rng.uniform(-100.0, +100.0, size=(N, D))

# Initial population.
swarm_t0 = Swarm([Particle(x) for x in X_t0])

# Create the StandardPSO object that will carry on the optimization.
test_PSO = StandardPSO(initial_swarm= swarm_t0,
                       obj_func= fun_Sphere,
                       x_min= -100.0, x_max= +100.0)

## Optimization process

In [13]:
'''
cProfile.run('test_PSO.run(max_it = 1000, f_tol = 1.0e-5, options = {"w": 0.5, "c1": 1.65, "c2": 1.65})',
             'output.prof')
'''
test_PSO.run(max_it = 300,
             f_tol = 1.0e-5,
             options = {"w": 0.5, "c1": 2.05, "c2": 2.05, "global_avg": True},
             reset_swarm = True,
             verbose = True)

Initial f_optimal = -2013.5144
Iteration:     1 -> f_optimal = -484.9911
Iteration:    31 -> f_optimal = -2.3239
Iteration:    61 -> f_optimal = -0.0071
Iteration:    91 -> f_optimal = -0.0043
Iteration:   121 -> f_optimal = -0.0000
Iteration:   151 -> f_optimal = -0.0000
Iteration:   181 -> f_optimal = -0.0000
Iteration:   211 -> f_optimal = -0.0000
Iteration:   241 -> f_optimal = -0.0000
Iteration:   271 -> f_optimal = -0.0000
Final f_optimal = -0.0000
run: elapsed time = 0.184 seconds.


## Final output

In [14]:
# Extract the optimal solution from the PSO.
optimal_solution = test_PSO.swarm.best_particle()

# Display the (final) optimal value.
print(f"Optimum Found: {optimal_solution.value:.6f}\n")

# Display each gene value separately.
for i, xi in enumerate(optimal_solution.position, start=1):
    print(f"x{i} = {xi:>10.6f}")
# _end_for_

Optimum Found: -0.000000

x1 =   0.000000
x2 =  -0.000000
x3 =  -0.000000
x4 =   0.000000
x5 =  -0.000000


In [15]:
# p = pstats.Stats('output.prof')

In [16]:
# p.sort_stats('cumtime').print_stats(20)

### End of file